In [1]:
from Librerias import *
from wazuhApi import *
from main import *
from tkinter import messagebox as MessageBox

ConnectionError: HTTPSConnectionPool(host='54.159.199.49', port=55000): Max retries exceeded with url: /security/user/authenticate (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000027641CEF1F0>: Failed to establish a new connection: [WinError 10060] Se produjo un error durante el intento de conexión ya que la parte conectada no respondió adecuadamente tras un periodo de tiempo, o bien se produjo un error en la conexión establecida ya que el host conectado no ha podido responder'))

In [1]:
# Creamos la ventana y titulo de la ventana

from Librerias import *
#from wazuhApi import *
from main import *
from tkinter import messagebox as MessageBox

ventana = tk.Tk()
ventana.iconbitmap('assets/wazuh.ico')
ventana.title("Los datos importantes")
ventana.configure(bg="skyblue")
menuBar(ventana)

# Dimensiones y posicion de inicio de la pantalla.
ventana.config(width=500, height=500)
altura = ventana.winfo_reqheight()
anchura = ventana.winfo_reqwidth()
altura_pantalla = ventana.winfo_screenheight()
anchura_pantalla = ventana.winfo_screenwidth()
ventana.geometry(
    f'{altura}x{anchura}+{int(anchura_pantalla/2-(anchura/2))}+{int(altura_pantalla/2-(altura/2))}')
ventana.resizable(False, False)

style=ttk.Style()
style.theme_use('alt')
style.configure("TScrollbar", background="blue")
widthListBox = anchura*.80
heightListBox = altura*.80
anchoSBY = anchura
altoSBX = altura

# Estilos y tamaños de letra
fontStyle_Labels = tkFont.Font(family="Lucida Grande", size=15)
fontStyle_Text = tkFont.Font(family="Helvetica bold", size=18)

Label(ventana, text="Hello World!", font=fontStyle_Text,anchor="center").pack(pady=10)

# Estilo para el texto
rangos = ["High","Medium","Critical","Low"]
# for i in rangos:
#     #print(f"Level: {i} -> ", json.dumps(apiTest.get_vul_by_crit(i), indent=4, sort_keys=True))
#     JP = (json.dumps(apiTest.get_vul_by_crit(i), indent=4, sort_keys=True))
#     # print(JP)  # JP es un String
#     datos_diccionario = json.loads(JP)

listbox = tk.Listbox(font=fontStyle_Text)
# value = datos_diccionario["data"]["affected_items"]
# listbox.insert(
#     tk.END, *(
#         f'ID: {value[i]["id"]}, \n Last Pick: {value[i]["lastKeepAlive"]}' 
#         for i in range(len(value))
#     )
# )

# LISTBOX positions and funtion about scrollbar
listbox.place(x=int(anchura*.9)-widthListBox, y=int(altura*.1), width=widthListBox, height=heightListBox)
scrollbar = ttk.Scrollbar(orient=tk.VERTICAL, command=listbox.yview)
scrollbar2 = ttk.Scrollbar(orient=tk.HORIZONTAL, command=listbox.xview)
listbox.config(yscrollcommand=scrollbar.set)
listbox.config(xscrollcommand=scrollbar2.set)
high(listbox)

# SCROLLBAR dimentions
scrollbar.place(x=anchoSBY-20, y=15, height=altura-40)
scrollbar2.place(x=5, y=altoSBX-20, width=anchoSBY-25)

ventana.mainloop()


Hello world!
good bye :)


In [36]:
"""def high(fontStyle_Text):
    print("Hello world!")

    rangos = ["High","Medium","Critical","Low"]
    listbox = tk.Listbox(font=fontStyle_Text)
    with open('JsonPrueba.json') as file:
        data = json.load(file)
        JP = (json.dumps(data, indent=4, sort_keys=True))
    datos_diccionario = json.loads(JP)
    value = datos_diccionario["data"]["affected_items"]
    listbox.insert(
        tk.END, *(
            f'ID: {value[i]["id"]}, \n Last Pick: {value[i]["lastKeepAlive"]}' 
            for i in range(len(value))
        )
    )"""

{
    "data": {
        "affected_items": [
            {
                "dateAdd": "2023-02-27T23:40:01Z",
                "id": "000",
                "ip": "127.0.0.1",
                "lastKeepAlive": "9999-12-31T23:59:59Z",
                "manager": "ip-172-31-32-91",
                "name": "ip-172-31-32-91",
                "node_name": "node01",
                "os": {
                    "arch": "x86_64",
                    "codename": "Jammy Jellyfish",
                    "major": "22",
                    "minor": "04",
                    "name": "Ubuntu",
                    "platform": "ubuntu",
                    "uname": "Linux |ip-172-31-32-91 |5.15.0-1028-aws |#32-Ubuntu SMP Mon Jan 9 12:28:07 UTC 2023 |x86_64",
                    "version": "22.04.1 LTS"
                },
                "registerIP": "127.0.0.1",
                "status": "active",
                "version": "Wazuh v4.3.10"
            },
            {
                "configSum": "ab73af4169

In [29]:
import tkinter as tk
my_w = tk.Tk()
my_w.geometry("350x200")  # Size of the window 
my_w.title("www.plus2net.com")  # Adding a title

options = tk.StringVar(my_w)
options.set("One") # default value

l1 = tk.Label(my_w,  text='Select One', width=10 )  
l1.grid(row=2,column=1) 

om1 =tk.OptionMenu(my_w, options, "HTML","PHP", "MySQL", "Python")
om1.grid(row=2,column=2) 

b1 = tk.Button(my_w,  text='Show Value', command=imprimirNuevo )  
b1.grid(row=2,column=3) 

str_out=tk.StringVar(my_w)
str_out.set("Output")

l2 = tk.Label(my_w,  textvariable=str_out, width=10 )  
l2.grid(row=2,column=4) 

def my_show():
    str_out.set(options.get())

my_w.mainloop()

TclError: can't invoke "button" command: application has been destroyed

In [12]:
import tkinter as tk
my_w = tk.Tk()
my_w.geometry("300x120")  # Size of the window 
my_w.title("www.plus2net.com")  # Adding a title

l3 = tk.Label(my_w,  text='Select One', width=15 )  
l3.grid(row=2,column=1)

my_list = ["PHP","MySQL","Python","HTML"]
options = tk.StringVar(my_w)
options.set(my_list[1]) # default value

om1 =tk.OptionMenu(my_w, options, *my_list)
om1.grid(row=2,column=2)
#print()
str_out=tk.StringVar(my_w)
str_out.set("Output")

l2 = tk.Label(my_w,  textvariable=str_out, width=10 )  
l2.grid(row=2,column=4) 

def my_show(*args):
    str_out.set(options.get())

options.trace('w',my_show)
my_w.mainloop()  # Keep the window open

In [43]:
apiTest = apiHandler()
apiTest.get_token()
jsonApi = apiTest.get_agents()
#print(apiTest.get_vul_by_crit("High"))
print(apiTest.get_vul_by_key("Abobe"))

https://54.145.241.208:55000/vulnerability/001?pretty=true&search=Abobe&select=name,condition,status,severity&limit=10
{'affected_items': [], 'total_affected_items': 0, 'total_failed_items': 0, 'failed_items': []}
https://54.145.241.208:55000/vulnerability/002?pretty=true&search=Abobe&select=name,condition,status,severity&limit=10
{'affected_items': [], 'total_affected_items': 0, 'total_failed_items': 0, 'failed_items': []}
https://54.145.241.208:55000/vulnerability/003?pretty=true&search=Abobe&select=name,condition,status,severity&limit=10
{'affected_items': [], 'total_affected_items': 0, 'total_failed_items': 0, 'failed_items': []}
https://54.145.241.208:55000/vulnerability/004?pretty=true&search=Abobe&select=name,condition,status,severity&limit=10
{'affected_items': [], 'total_affected_items': 0, 'total_failed_items': 0, 'failed_items': []}
https://54.145.241.208:55000/vulnerability/005?pretty=true&search=Abobe&select=name,condition,status,severity&limit=10
{'affected_items': [], 't

Exception: Error obtaining response: {'title': 'Resource Not Found', 'detail': 'Agent does not exist', 'remediation': 'Please, use `GET /agents?select=id,name` to find all available agents', 'dapi_errors': {'node01': {'error': 'Agent does not exist'}}, 'error': 1701}

In [1]:
from Librerias import *
from wazuhApi import *

''' Sacar el numero total de vulnerabilidades por severidad '''
rangos = ["High","Medium","Critical","Low"]
for i in rangos:
    print(f"Level: {i} -> ", json.dumps(apiTest.get_vul_by_crit(i), indent=4, sort_keys=True))

''' Sacar vulnerabilidades por palabra clave con limite default de 10 por agente '''
#print("Palabra: ",apiTest.get_vul_by_key("Adobe"))
#print(json.dumps(apiTest.get_vul_by_key("Adobe"), indent=4, sort_keys=True))

HIGH: 1616
https://54.145.241.208:55000/vulnerability/001?pretty=true&search=Adobe&select=name,condition,status,severity&limit=10
{'affected_items': [], 'total_affected_items': 0, 'total_failed_items': 0, 'failed_items': []}
https://54.145.241.208:55000/vulnerability/002?pretty=true&search=Adobe&select=name,condition,status,severity&limit=10
{'affected_items': [], 'total_affected_items': 0, 'total_failed_items': 0, 'failed_items': []}
https://54.145.241.208:55000/vulnerability/003?pretty=true&search=Adobe&select=name,condition,status,severity&limit=10
{'affected_items': [], 'total_affected_items': 0, 'total_failed_items': 0, 'failed_items': []}
https://54.145.241.208:55000/vulnerability/004?pretty=true&search=Adobe&select=name,condition,status,severity&limit=10
{'affected_items': [], 'total_affected_items': 0, 'total_failed_items': 0, 'failed_items': []}
https://54.145.241.208:55000/vulnerability/005?pretty=true&search=Adobe&select=name,condition,status,severity&limit=10
{'affected_ite

' Sacar vulnerabilidades por palabra clave con limite default de 10 por agente '

In [9]:
# ENTRY
from tkinter import *

# Configuración de la raíz
root = Tk()

label = Label(root, text="Nombre muy largo")
label.grid(row=0, column=0, sticky="w", padx=5, pady=5)

entry = Entry(root)
entry.grid(row=0, column=1, padx=5, pady=5)
entry.config(justify="right", state="normal")

label2 = Label(root, text="Contraseña")
label2.grid(row=1, column=0, sticky="w", padx=5, pady=5)

entry2 = Entry(root)
entry2.grid(row=1, column=1, padx=5, pady=5)
entry2.config(justify="center", show="*")

# Finalmente bucle de la aplicación
root.mainloop()